In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('Data\process_taxis.parquet',engine='pyarrow')

In [6]:
df['trip_duration'] = (df['dropoff_datetime'] - df['pickup_datetime']).dt.total_seconds()

In [7]:
df.tail(3)

,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_price,service,trip_duration
57785405,2024-07-01 00:05:23,2024-07-01 00:15:02,5.0,2.07,263,151,26.70,0,579.0
57785406,2026-06-26 23:26:55,2026-06-26 23:47:51,1.0,10.96,138,233,85.25,0,1256.0
57785407,2026-06-26 23:53:12,2026-06-27 20:59:10,1.0,0.56,233,264,16.20,0,75958.0
